Teaser
===========

In [ ]:
import sys
import os
sys.path.append(os.path.realpath(".."))
from preamble import *
recompute = False


In [ ]:
# Load wireframes
wireframes = []
renderings = []
res = 400
collection = 25
vp = 11
scene = "nefertiti"

for j, method in enumerate(["base", "naive", "reg", "ours", "remesh"]):
    # Render with wireframe
    filename = os.path.join(basename, f"res_{method}_wireframe.png")
    if recompute or not os.path.exists(filename):
        blender_render(os.path.join(basename, f"res_{method}.ply"), 
                       basename, 
                       collection,
                       vp,
                       res=res,
                       ours=(j>2),
                       baseline=(j in [1,2]),
                       wireframe=True,
                       t=0.004)
    wireframes.append(plt.imread(filename))
    # Render without wireframe
    if j>0:
        filename = os.path.join(basename, f"res_{method}_smooth.png")
        if recompute or not os.path.exists(filename):
            blender_render(os.path.join(basename, f"res_{method}.ply"), 
                           basename, 
                           collection,
                           vp,
                           res=res,
                           ours=(j>2),
                           baseline=(j in [1,2]),
                           wireframe=False,
                           t=0.004)
        renderings.append(plt.imread(filename))

filename = os.path.join(basename, "ref_smooth.png")
if not os.path.exists(filename) or recompute:
    blender_render(os.path.join(basename, f"ref.ply"), 
               basename, 
               collection,
               vp,
               res=res,
               t=0.004)
renderings.append(plt.imread(filename))

In [ ]:
base_size = 10
n_rows = 1
n_cols = len(wireframes)+1
H,W,_ = wireframes[0].shape
ratio = 0.3

w = int(ratio*W)
x_offset = -int(0.02*w)
aspect = w/H
inset_w = int(0.12*H)

fontsize = 36
widths = np.ones(n_cols)

fig = plt.figure(1, figsize=(n_cols * base_size* aspect, n_rows * base_size*1.1), constrained_layout=True)
gs = fig.add_gridspec(n_rows,  n_cols, wspace=0.075,hspace=0.0, width_ratios=widths)

sns.set_style("white")

kw = dict(ha="center", va="center", fontsize=fontsize, color="darkgrey", rotation=45)

# Indices for the diagonal split, J>I above the diagonal, and J<I below
J,I = np.meshgrid(np.linspace(0,W/w,W)  + (w-W)/(2*w), np.linspace(0,1,H))

for i, name in enumerate(["(a) Initial State", "(b) Naïve", "(c) Regularized", "(d) Ours", " (e) Ours (with remeshing)", " (f) Reference"]):
    ax = fig.add_subplot(gs[i])
    ax.set_xlabel(name, fontsize=fontsize, y=-0.2)
    if i < n_cols-1:
        wf = wireframes[i].copy()

        if i == 0:
            ax.imshow(wireframes[i][:, (W-w)//2+x_offset:(W+w)//2+x_offset],)
        else:
            im = np.ones_like(renderings[i-1])
            im[I>J] = wf[I>J]
            im[I<J] = renderings[i-1][I<J]
            im = im*im[..., -1][..., None] + (1-im[..., -1][..., None])
            im_cropped = np.flip(im[:, (W-w)//2+x_offset:(W+w)//2+x_offset],0)
            
            ax.imshow(im_cropped, origin='lower',)
            
            axins = ax.inset_axes([0.6, 0, 0.4, 0.4])
            axins.imshow(im_cropped, origin='lower',)
            axins.tick_params(axis='both', which='both', labelleft=False, labelbottom=False)
            x0 = int(w*0.21)
            y0 = int(0.45*H)
            x1, x2, y1, y2 = (x0, x0+inset_w, y0, y0+inset_w)
            axins.set_xlim(x1, x2)
            axins.set_ylim(y1, y2)
            ax.indicate_inset_zoom(axins, edgecolor="black", alpha=1)

    else:
        im = renderings[i-1][:, (W-w)//2+x_offset:(W+w)//2+x_offset]
        im = im*im[..., -1][..., None] + (1-im[..., -1][..., None])
        ax.imshow(im,)

    ax.tick_params(
        axis='both',
        which='both',
        labelleft=False,
        labelbottom=False)
    sns.despine(ax=ax, left=True, bottom=True)

plt.savefig(os.path.join(basename, "teaser.pdf"), format='pdf', dpi=300, bbox_inches='tight', pad_inches=0.005)